In [1]:
import glob
import os
from os.path import join

import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
def get_features_filtered_by_label(features, labels, selected_label):
    return [feature for feature, label in zip(features, labels) if label == selected_label]

# check normalize
def get_k_main_eigenvectors_svd(features, k=1):
    _, _, vh = np.linalg.svd(normalize(features), full_matrices=False)
    return vh[:k]

def get_k_main_eigenvectors(X, k=1):
    m1 = X.mean(axis=0)
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0])
    w, v = np.linalg.eigh(m2)
    return v[:, -k:].T

def get_name(path):
    return os.path.splitext(os.path.split(path)[1])[0]

def test(eigen_female, eigen_male, features_test, is_correct_function, debug=False):
    counter = 0
    for feature in features_test:
        eigen_test = get_k_main_eigenvectors(feature, k=num_eigenvectors)
        dist_to_female = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_female, eigen_test)])
        dist_to_male = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_male, eigen_test)])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

# -----------------------------------------------------------------
# def normalize(x):
#     return x - x.mean()

def normalize(x):
    return x / np.linalg.norm(x, ord=2, axis=1, keepdims=True)

In [3]:
paths = sorted(glob.glob('datasets/khanty_4/*.wav'))

---

In [4]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:15<00:00,  3.83it/s]


In [5]:
for random_state in [120, 5544, 33, 23, 997, 719, 581]:
    X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list, test_size=0.33,
                                                        random_state=random_state, stratify=labels_list)

    num_eigenvectors = 1 # num_eigenvectors > 1 doesn't improve accuracy; need to improve inference

    features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
    eigen_female = get_k_main_eigenvectors(features, k=num_eigenvectors)

    features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
    eigen_male = get_k_main_eigenvectors(features, k=num_eigenvectors)

    features_test = get_features_filtered_by_label(X_test, y_test, 'F')
    is_correct_function = lambda x, y: x < y
    accuracy_female = test(eigen_female, eigen_male, features_test, is_correct_function)

    features_test = get_features_filtered_by_label(X_test, y_test, 'M')
    is_correct_function = lambda x, y: x > y
    accuracy_male = test(eigen_female, eigen_male, features_test, is_correct_function)

    print(f'f: {accuracy_female:.3f}, m: {accuracy_male:.3f}')

f: 0.765, m: 0.871
f: 0.765, m: 0.914
f: 0.784, m: 0.871
f: 0.814, m: 0.929
f: 0.696, m: 0.871
f: 0.794, m: 0.914
f: 0.804, m: 0.900


---

In [6]:
# features_test = get_features_filtered_by_label(X_test, y_test, 'F')
# is_correct_function = lambda x, y: x < y

# counter = 0
# for feature in features_test:
#     feature = normalize(feature)
#     dist_to_female = sum([min([np.square(x - etr).sum() for etr in eigen_female]) for x in feature])
#     dist_to_male = sum([min([np.square(x - etr).sum() for etr in eigen_male]) for x in feature])
#     correct = is_correct_function(dist_to_female, dist_to_male)
#     if correct:
#         counter += 1
# #     print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
# print(f'{counter/len(features_test):.3f}')

In [7]:
# features_test = get_features_filtered_by_label(X_test, y_test, 'M')
# is_correct_function = lambda x, y: x > y

# counter = 0
# for feature in features_test:
#     feature = normalize(feature)
#     dist_to_female = sum([min([np.square(x - etr).sum() for etr in eigen_female]) for x in feature])
#     dist_to_male = sum([min([np.square(x - etr).sum() for etr in eigen_male]) for x in feature])
#     correct = is_correct_function(dist_to_female, dist_to_male)
#     if correct:
#         counter += 1
# #         print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
# print(f'{counter/len(features_test):.3f}')